## 9. High Accuracy CNN for MNIST
_Exercise: Build your own CNN from scratch and try to achieve the highest possible accuracy on MNIST._

The following model uses

- 2 convolutional layers,

- followed by 1 pooling layer,

- then dropout 25%,

- then a dense layer,

- another dropout layer but with 50% dropout,

- and finally the output layer.

It reaches about 99.2% accuracy on the test set. This places this model roughly in the top 20% in the [MNIST Kaggle competition](https://www.kaggle.com/c/digit-recognizer/) (if we ignore the models with an accuracy greater than 99.79% which were most likely trained on the test set, as explained by Chris Deotte in [this post](https://www.kaggle.com/c/digit-recognizer/discussion/61480)).

Can you do better?

To reach 99.5 to 99.7% accuracy on the test set, you need to add

- image augmentation,

- batch norm, use a learning schedule such as 1-cycle,

- and possibly create an ensemble.

In [9]:
import warnings
warnings.filterwarnings("ignore")

In [10]:
import numpy as np

import tensorflow as tf
from tensorflow import keras

In [11]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()

X_train_full = X_train_full / 255
X_test = X_test / 255

X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

11490434/11490434 [==============================] - 0s 0us/step


In [12]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [13]:
# Model building
model = keras.models.Sequential([
    keras.layers.Conv2D(32, kernel_size=3, padding="same", activation="relu"),
    keras.layers.Conv2D(64, kernel_size=3, padding="same", activation="relu"),
    keras.layers.MaxPool2D(),
    keras.layers.Flatten(),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])

In [14]:
# Model training and testing
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
model.evaluate(X_test, y_test)

Epoch 1/10
1719/1719 [==============================] - 217s 121ms/step - loss: 0.1946 - accuracy: 0.9410 - val_loss: 0.0450 - val_accuracy: 0.9876
Epoch 2/10
1719/1719 [==============================] - 205s 119ms/step - loss: 0.0807 - accuracy: 0.9759 - val_loss: 0.0413 - val_accuracy: 0.9882
Epoch 3/10
1719/1719 [==============================] - 204s 119ms/step - loss: 0.0613 - accuracy: 0.9815 - val_loss: 0.0375 - val_accuracy: 0.9906
Epoch 4/10
1719/1719 [==============================] - 207s 121ms/step - loss: 0.0487 - accuracy: 0.9851 - val_loss: 0.0323 - val_accuracy: 0.9908
Epoch 5/10
1719/1719 [==============================] - 206s 120ms/step - loss: 0.0421 - accuracy: 0.9871 - val_loss: 0.0401 - val_accuracy: 0.9906
Epoch 6/10
1719/1719 [==============================] - 204s 119ms/step - loss: 0.0351 - accuracy: 0.9889 - val_loss: 0.0367 - val_accuracy: 0.9924
Epoch 7/10
1719/1719 [==============================] - 205s 119ms/step - loss: 0.0319 - accuracy: 0.9896 - val_

[0.02948763780295849, 0.9925000071525574]

## 10.  Use transfer learning for large image classification

_Exercise: Use transfer learning for large image classification, going through these steps:_

* _Create a training set containing at least 100 images per class. For example, you could classify your own pictures based on the location (beach, mountain, city, etc.), or alternatively you can use an existing dataset (e.g., from TensorFlow Datasets)._
* _Split it into a training set, a validation set, and a test set._
* _Build the input pipeline, including the appropriate preprocessing operations, and optionally add data augmentation._
* _Fine-tune a pretrained model on this dataset._

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Load the dataset
dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)

# Define the classes
num_classes = info.features['label'].num_classes

# Define the training, validation, and test split sizes
train_split = 0.7
val_split = 0.15
test_split = 0.15

# Calculate the number of examples for each split
num_examples = info.splits['train'].num_examples
num_train_examples = int(train_split * num_examples)
num_val_examples = int(val_split * num_examples)
num_test_examples = num_examples - num_train_examples - num_val_examples

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [3]:
# Split the dataset
train_dataset = dataset['train'].take(num_train_examples)
val_dataset = dataset['train'].skip(num_train_examples).take(num_val_examples)
test_dataset = dataset['train'].skip(num_train_examples + num_val_examples).take(num_test_examples)

In [4]:
# Define preprocessing function
def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224))  # Resize image to expected input size for MobileNetV2
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)  # Preprocess the input image
    return image, label

# Apply preprocessing to datasets
train_dataset = train_dataset.map(preprocess_image)
val_dataset = val_dataset.map(preprocess_image)
test_dataset = test_dataset.map(preprocess_image)

# Define batch size
batch_size = 32

# Shuffle and batch the datasets
train_dataset = train_dataset.shuffle(buffer_size=1000).batch(batch_size)
val_dataset = val_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

In [5]:
# Load pretrained MobileNetV2 model
pretrained_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pretrained layers
pretrained_model.trainable = False

9406464/9406464 [==============================] - 0s 0us/step


In [6]:
# Define the new classification head
model = tf.keras.Sequential([
    pretrained_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [8]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_dataset, epochs=10, validation_data=val_dataset)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_dataset)
print("Test Accuracy:", test_acc)

Epoch 1/40
81/81 [==============================] - 98s 1s/step - loss: 0.1179 - accuracy: 0.9704 - val_loss: 0.2791 - val_accuracy: 0.9000
Epoch 2/40
81/81 [==============================] - 99s 1s/step - loss: 0.0974 - accuracy: 0.9798 - val_loss: 0.2923 - val_accuracy: 0.8982
Epoch 3/40
81/81 [==============================] - 96s 1s/step - loss: 0.0870 - accuracy: 0.9813 - val_loss: 0.2885 - val_accuracy: 0.9000
Epoch 4/40
81/81 [==============================] - 130s 2s/step - loss: 0.0758 - accuracy: 0.9872 - val_loss: 0.3028 - val_accuracy: 0.8964
Epoch 5/40
81/81 [==============================] - 96s 1s/step - loss: 0.0721 - accuracy: 0.9887 - val_loss: 0.2936 - val_accuracy: 0.8982
Epoch 6/40
81/81 [==============================] - 95s 1s/step - loss: 0.0604 - accuracy: 0.9930 - val_loss: 0.2835 - val_accuracy: 0.9018
Epoch 7/40
81/81 [==============================] - 97s 1s/step - loss: 0.0540 - accuracy: 0.9942 - val_loss: 0.2914 - val_accuracy: 0.8964
Epoch 8/40
81/81 [=